# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [ ]:
#импортируем все необходимые для работы библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st

In [ ]:
#считаем три датасета и сохраним их в отдельных переменных
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
#выведем первые 10 строк трёх датафреймов на экран, чтобы ознакомиться с данными
data_0.head(10)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [ ]:
data_1.head(10)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [ ]:
data_2.head(10)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [ ]:
#теперь изучим основную информацию о трёх датафреймах
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Все три датафрейма содержат информацию об:
- id — уникальном идентификаторе скважины,
- f0, f1, f2 — трёх признаках точек,
- product — объёме запасов в скважине (тыс. баррелей)

Пропусков ни в одном из датафреймов нет. Проверим, есть ли в них явные дубликаты.

In [ ]:
data_0.duplicated().sum()

0

In [ ]:
data_1.duplicated().sum()

0

In [ ]:
data_2.duplicated().sum()

0

Явных дубликатов также нет.

Для обучения модели признаками будут являться столбцы f0, f1, f2, а целевым признаком - product. Столбец id с уникалным идентификатором скважины содержит категориальные данные, которые никак не влияют на целевой признак (так как это просто "имя" скважины), но могут запутать нашу модель. Удалим этот столбец во всех датафреймах.

In [ ]:
data_0 = data_0.drop(['id'], axis=1)
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)

In [ ]:
#посмотрим на размеры получившихся датафреймов
print(data_0.shape)
print(data_1.shape)
print(data_2.shape)

(100000, 4)
(100000, 4)
(100000, 4)


Столбец id успешно удалён, все нужные столбцы и строки на месте. В оставшихся столбцах содержатся числовые данные, поэтому преобразование данных не требуется. Также мы видим, что признаки f0, f1, f2 единообразны, поэтому масштабирование нам также не потребуется.

**Вывод:** мы изучили исходные данные и подготовили их для дальнейшего машинного обучения: убедились в отсутствии пропусков и дубликатов, удалили лишний столбец и убедились в отсутсвии необходимости преобразования и масштабирования данных.

## Обучение и проверка модели

Разделим данные на обучающую и валидационную выборки в соотношении 75:25.

In [ ]:
#укажем признаки и целевой признак для каждого датафрейма
features_0 = data_0.drop('product', axis=1)
target_0 = data_0['product']
features_1 = data_1.drop('product', axis=1)
target_1 = data_1['product']
features_2 = data_2.drop('product', axis=1)
target_2 = data_2['product']

По условиям задачи, для обучения модели подходит только линейная регрессия. Обучим три отдельные модели линейной регрессии для каждого датафрейма.

In [ ]:
#разбиваем данные на обучающие и тестовые выборки
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0,
                                                                            target_0,
                                                                            test_size=0.25,
                                                                            random_state=12345
                                                                           )

In [ ]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1,
                                                                            target_1,
                                                                            test_size=0.25,
                                                                            random_state=12345
                                                                           )

In [ ]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2,
                                                                            target_2,
                                                                            test_size=0.25,
                                                                            random_state=12345
                                                                           )

Проверим размеры обучающих и валиационных выборок.

In [ ]:
print('Размеры обучающих выборок:')
print('"0":')
print(features_train_0.shape)
print(target_train_0.shape)
print('"1":')
print(features_train_1.shape)
print(target_train_1.shape)
print('"2":')
print(features_train_2.shape)
print(target_train_2.shape)
print()
print('Размеры валидационных выборок:')
print('"0":')
print(features_valid_0.shape)
print(target_valid_0.shape)
print('"1":')
print(features_valid_1.shape)
print(target_valid_1.shape)
print('"2":')
print(features_valid_2.shape)
print(target_valid_2.shape)

Размеры обучающих выборок:
"0":
(75000, 3)
(75000,)
"1":
(75000, 3)
(75000,)
"2":
(75000, 3)
(75000,)

Размеры валидационных выборок:
"0":
(25000, 3)
(25000,)
"1":
(25000, 3)
(25000,)
"2":
(25000, 3)
(25000,)


Данные разделены верно. Теперь обучим три модели линейной регрессии.

In [ ]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)

LinearRegression()

In [ ]:
#сделаем предсказания по каждой модели и сохраним их в отдельных переменных
predictions_valid_0 = model_0.predict(features_valid_0)
predictions_valid_1 = model_1.predict(features_valid_1)
predictions_valid_2 = model_2.predict(features_valid_2)

#и рассчитаем значение RMSE для каждой модели
rmse_0 = mean_squared_error(target_valid_0, predictions_valid_0) ** 0.5
rmse_1 = mean_squared_error(target_valid_1, predictions_valid_1) ** 0.5
rmse_2 = mean_squared_error(target_valid_2, predictions_valid_2) ** 0.5

#выведем на экран средний запас предсказанного сырья и RMSE для каждой модели
print('Модель "0":')
print('Средний запас предсказанного сырья:', predictions_valid_0.mean())
print('RMSE:', rmse_0)
print()
print('Модель "1":')
print('Средний запас предсказанного сырья:', predictions_valid_1.mean())
print('RMSE:', rmse_1)
print()
print('Модель "2":')
print('Средний запас предсказанного сырья:', predictions_valid_2.mean())
print('RMSE:', rmse_2)

Модель "0":
Средний запас предсказанного сырья: 92.59256778438035
RMSE: 37.5794217150813

Модель "1":
Средний запас предсказанного сырья: 68.728546895446
RMSE: 0.893099286775617

Модель "2":
Средний запас предсказанного сырья: 94.96504596800489
RMSE: 40.02970873393434


**Вывод:** мы разделили данные на обучающую и валидационную выборки в соотношении 75:25. Согласно условиям задачи, мы выбрали модель линейной регрессии и обучили отдельные три модели для каждого датафрейма. Затем мы рассчитали средний запас предсказанного сырья и значение RMSE. Самый большой средний запас предсказанного сырья показала модель "2" - 94.96504596800489 тыс. баррелей, однако и RMSE у этой модели самый большой - 40.02970873393434 (тыс. баррелей). Наименьший средний запас предсказанного сырья показала модель "1" - 68.728546895446 тыс. баррелей, но эта модель гораздо точнее. Её показатель RMSE равен 0.893099286775617 (тыс. баррелей).

## Подготовка к расчёту прибыли

Сохраним все ключевые значения для расчётов в отдельных переменных.

In [ ]:
total_budget = 10000000000 #бюджет на разработку всех скважин в регионе
rev = 450000 #доход с каждой единицы продукта (1 тыс. баррелей)
all_wells = 500 #при разведке региона исследуют 500 точек
best_wells = 200 #в регионе выбирают 200 лучших скважин для разработки

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [ ]:
print('Объём сырья для безубыточной разработки новой скважины:', total_budget / rev / best_wells, 'тыс. баррелей')

Объём сырья для безубыточной разработки новой скважины: 111.11111111111111 тыс. баррелей


Сравним полученный объём сырья со средним запасом в каждом регионе.

In [ ]:
print('Средний запас сырья в каждом регионе:')
print('Регион "0":', data_0['product'].mean(), 'тыс. баррелей')
print('Регион "1":', data_1['product'].mean(), 'тыс. баррелей')
print('Регион "2":', data_2['product'].mean(), 'тыс. баррелей')

Средний запас сырья в каждом регионе:
Регион "0": 92.50000000000001 тыс. баррелей
Регион "1": 68.82500000000002 тыс. баррелей
Регион "2": 95.00000000000004 тыс. баррелей


**Вывод:** мы выяснили, что минимальный объём сырья для безубыточной разработки новой скважины равен 111 тыс. баррелей. Во всех трёх регионах средний запас сырья ниже этого значения: в регионе "0" 92.5 тыс. баррелей, в регионе "1" 68.8 тыс. баррелей, в регионе "2" 95 тыс. баррелей - то есть во всех регионах вести разработку рисковано. Мы смотрели на среднее значение запаса сырья в каждом регионе - оно складывается как из скважин с совсем небольшим запасом, так и из скважин с богатым запасом, большим, чем минимальное необходимое количество. В следующем шаге применим технику Bootstrap, чтобы случайным образом отбирать скважины и рассчитывать для них прибыль и риски.

## Расчёт прибыли и рисков

Напишем функцию для расчёта прибыли.

In [ ]:
def revenue(target, predictions):
    probs_sorted = pd.Series(predictions).sort_values(ascending=False) #выбираем значения предсказаний с лучшими показателями вверху списка
    selected = target[probs_sorted.index][:best_wells] #выбираем 200 лучших скважин
    return rev * selected.sum() - total_budget #считаем прибыль

Теперь применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [ ]:
def bootstrap(target, predictions):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target = target.reset_index(drop=True)
        target_subsample = target.sample(n = all_wells, replace = True, random_state = state)
        predictions_subsample = pd.Series(predictions)[target_subsample.index]

        values.append(revenue(target_subsample, predictions_subsample))

    values = pd.Series(values)
    return values

Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. Создадим функции, которые рассчитывают эти значения.

In [ ]:
def mean_revenue(values):
    print("Средняя выручка:", values.mean())

def conf_interval_95(values):
    print("95%-й доверительный интервал:", values.quantile(0.025), '-', values.quantile(0.975))

def risk_of_loss(values):
    print("Риск убытков:", st.percentileofscore(values, 0), '%')

Найдем распределение возможной прибыли для каждого региона.

In [ ]:
values_0 = bootstrap(target_valid_0, predictions_valid_0)
values_1 = bootstrap(target_valid_1, predictions_valid_1)
values_2 = bootstrap(target_valid_2, predictions_valid_2)

Теперь рассчитаем все значения для каждого региона и выведем на экран:

In [ ]:
print('Для региона "0":')
print(mean_revenue(values_0))
print(conf_interval_95(values_0))
print(risk_of_loss(values_0))
print()
print('Для региона "1":')
print(mean_revenue(values_1))
print(conf_interval_95(values_1))
print(risk_of_loss(values_1))
print()
print('Для региона "2":')
print(mean_revenue(values_2))
print(conf_interval_95(values_2))
print(risk_of_loss(values_2))

Для региона "0":
Средняя выручка: 425938526.91059244
None
95%-й доверительный интервал: -102090094.83793654 - 947976353.3583689
None
Риск убытков: 6.0 %
None

Для региона "1":
Средняя выручка: 515222773.4432899
None
95%-й доверительный интервал: 68873225.37050176 - 931547591.2570494
None
Риск убытков: 1.0 %
None

Для региона "2":
Средняя выручка: 435008362.7827556
None
95%-й доверительный интервал: -128880547.32978901 - 969706954.1802661
None
Риск убытков: 6.4 %
None


**Вывод:** для регионов "0" и "2" левая граница доверительного интервала отрицательная, и риск убытков достаточно высокий - 6% и 6,4% соответственно. Для региона "1" доверительный интервал лежит в положительной зоне, и риск убытков наименьший - 1%. Средняя выручка самая большая для региона "1" - 515222773.44 рублей, за нем следует регион "2" - 435008362.78 рублей и затем регион "0" - 425938526.91 рублей. Также по условиям задачи нам подходят только те регионы, в которых уровень риска убытков будет менее 2,5%. Таким образом, **для разработки следует выбрать регион "1"** - для него средняя выручка самая высокая, а риск наименьший.

## Вывод

Мы провели исследование по выбору локации для скважины для компании «ГлавРосГосНефть».

1) Мы изучили исходные данные и подготовили их для дальнейшего машинного обучения: убедились в отсутствии пропусков и дубликатов, удалили лишний столбец и убедились в отсутсвии необходимости преобразования и масштабирования данных.

2) Мы разделили данные на обучающую и валидационную выборки в соотношении 75:25. Согласно условиям задачи, мы выбрали модель линейной регрессии и обучили отдельные три модели для каждого датафрейма. Затем мы рассчитали средний запас предсказанного сырья и значение RMSE. Самый большой средний запас предсказанного сырья показала модель "2" - 94.96504596800489 тыс. баррелей, однако и RMSE у этой модели самый большой - 40.02970873393434 (тыс. баррелей). Наименьший средний запас предсказанного сырья показала модель "1" - 68.728546895446 тыс. баррелей, но эта модель гораздо точнее. Её показатель RMSE равен 0.893099286775617 (тыс. баррелей).

3) Мы выяснили, что минимальный объём сырья для безубыточной разработки новой скважины равен 111 тыс. баррелей. Во всех трёх регионах средний запас сырья ниже этого значения: в регионе "0" 92.5 тыс. баррелей, в регионе "1" 68.8 тыс. баррелей, в регионе "2" 95 тыс. баррелей - то есть во всех регионах вести разработку рисковано. Мы смотрели на среднее значение запаса сырья в каждом регионе - оно складывается как из скважин с совсем небольшим запасом, так и из скважин с богатым запасом, большим, чем минимальное необходимое количество. Поэтому в следующем шаге мы решили применить технику Bootstrap, чтобы случайным образом отбирать скважины и рассчитывать для них прибыль и риски.

4) Применив Bootstrap, мы вычислили, что для регионов "0" и "2" левая граница доверительного интервала отрицательная, и риск убытков достаточно высокий - 6% и 6,4% соответственно. Для региона "1" доверительный интервал лежит в положительной зоне, и риск убытков наименьший - 1%. Средняя выручка самая большая для региона "1" - 515222773.44 рублей, за нем следует регион "2" - 435008362.78 рублей и затем регион "0" - 425938526.91 рублей. Также по условиям задачи нам подходят только те регионы, в которых уровень риска убытков будет менее 2,5%.

**Таким образом, для разработки следует выбрать регион "1" - для него средняя выручка самая высокая - 515222773.44 рублей, а риск убытков наименьший - 1%.**